In [18]:
import pickle
import pandas as pd
import matplotlib.pyplot as plt

In [19]:
# read the pickle file
with open('../tracker_stubs/ball_detections.pkl', 'rb') as f:
    ball_positions = pickle.load(f)

In [ ]:
ball_positions = [x.get(1, []) for x in ball_positions]
df_ball_positions = pd.DataFrame(ball_positions, columns=['x1', 'y1', 'x2', 'y2'])
df_ball_positions = df_ball_positions.interpolate()
df_ball_positions = df_ball_positions.bfill()

In [ ]:
df_ball_positions.head()

In [ ]:
df_ball_positions['mid_y'] = (df_ball_positions['y1'] + df_ball_positions['y2']) / 2  # center of the ball
df_ball_positions['mid_y_rolling_mean'] = df_ball_positions['mid_y'].rolling(
    window=5, min_periods=1, center=False).mean()  # rolling mean of the center of the ball

In [ ]:
# plot mid_y_rolling_mean
df_ball_positions['mid_y_rolling_mean'].plot()

In [ ]:
df_ball_positions['delta_y'] = df_ball_positions['mid_y_rolling_mean'].diff()  # this will subtract 2 consecutive values

In [ ]:
df_ball_positions['delta_y'].plot()

In [ ]:
df_ball_positions['ball_hit'] = 0

minimum_change_frames_for_hit = 25  # minimum number of frames for a hit to be interpreted as a hit
for i in range(1, len(df_ball_positions) - int(minimum_change_frames_for_hit * 1.2)):
    negative_position_change = df_ball_positions['delta_y'].iloc[i] > 0 and df_ball_positions['delta_y'].iloc[i + 1] < 0
    positive_position_change = df_ball_positions['delta_y'].iloc[i] < 0 and df_ball_positions['delta_y'].iloc[i + 1] > 0

    if negative_position_change or positive_position_change:
        change_count = 0
        for change_frame in range(i + 1, i + int(minimum_change_frames_for_hit * 1.2) + 1):
            negative_position_change_following_frame = df_ball_positions['delta_y'].iloc[i] > 0 and \
                                                       df_ball_positions['delta_y'].iloc[change_frame] < 0
            positive_position_change_following_frame = df_ball_positions['delta_y'].iloc[i] < 0 and \
                                                       df_ball_positions['delta_y'].iloc[change_frame] > 0

            if negative_position_change and negative_position_change_following_frame:
                change_count += 1
            elif positive_position_change and positive_position_change_following_frame:
                change_count += 1

        if change_count > minimum_change_frames_for_hit - 1:
            df_ball_positions['ball_hit'].iloc[i] = 1

frame_nums_with_ball_hits = df_ball_positions[df_ball_positions['ball_hit'] == 1].index.tolist()

In [ ]:
df_ball_positions.head()

In [ ]:
df_ball_positions[df_ball_positions['ball_hit'] == 1]

In [ ]:
frame_nums_with_ball_hits = df_ball_positions[df_ball_positions['ball_hit'] == 1].index.tolist()
frame_nums_witl_ball_hits